In [ ]:
# Install required libraries first
!pip install gradio #for user interface
!pip install bcrypt #for secure password hashing
!pip install zaproxy #for security testing

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 MB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.6/322.6 kB 16.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 4.8 MB/s eta 0:00:00


1. Database Setup (Run Once)

In [ ]:
import sqlite3

def create_database():
    conn = sqlite3.connect('disaster_relief.db')
    cursor = conn.cursor()

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS users (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            username TEXT UNIQUE NOT NULL,
            password TEXT NOT NULL,
            role TEXT NOT NULL
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS contacts (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            name TEXT NOT NULL,
            age INTEGER,
            location TEXT,
            notes TEXT
        )
    ''')

    cursor.execute('''
        CREATE TABLE IF NOT EXISTS security_log (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            action TEXT NOT NULL,
            timestamp DATETIME DEFAULT CURRENT_TIMESTAMP
        )
    ''')

    conn.commit()
    conn.close()

create_database()
print("Database initialized successfully.")

Database initialized successfully.


2. User Authentication (Login, Register)

In [ ]:
import hashlib

def register_user(username, password, role):
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    conn = sqlite3.connect('disaster_relief.db')
    cursor = conn.cursor()
    try:
        cursor.execute("INSERT INTO users (username, password, role) VALUES (?, ?, ?)",
                       (username, hashed_password, role))
        conn.commit()
    except sqlite3.IntegrityError:
        return "Username already exists."
    finally:
        conn.close()
    return "User registered successfully."

def authenticate_user(username, password):
    hashed_password = hashlib.sha256(password.encode()).hexdigest()
    conn = sqlite3.connect('disaster_relief.db')
    cursor = conn.cursor()
    cursor.execute("SELECT role FROM users WHERE username = ? AND password = ?",
                   (username, hashed_password))
    result = cursor.fetchone()
    conn.close()
    return result[0] if result else None

3. Security Logging

In [ ]:
def log_security_event(action):
    conn = sqlite3.connect('disaster_relief.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO security_log (action) VALUES (?)", (action,))
    conn.commit()
    conn.close()

4. Frontend User Interface (Using Gradio)

In [ ]:
import sqlite3
import gradio as gr
from werkzeug.security import generate_password_hash, check_password_hash

assignments = {}

# --- Core Functions ---

def register_user(username, password, role):
    hashed_password = generate_password_hash(password)
    conn = sqlite3.connect('disaster_relief.db')
    c = conn.cursor()
    try:
        c.execute("INSERT INTO users (username, password, role) VALUES (?, ?, ?)", (username, hashed_password, role))
        conn.commit()
        return "User registered successfully!"
    except sqlite3.IntegrityError:
        return "Username already exists!"
    finally:
        conn.close()

def login_user(username, password):
    conn = sqlite3.connect('disaster_relief.db')
    c = conn.cursor()
    c.execute("SELECT username, password, role FROM users WHERE username = ?", (username,))
    user = c.fetchone()
    conn.close()

    if user and check_password_hash(user[1], password):
        return f"Login successful as {user[2]}", user[2], user[0], True
    else:
        return "Login failed. Try again.", None, None, False

def show_dashboard(role):
    if role == 'volunteer':
        return "Volunteer Dashboard"
    elif role == 'staff':
        return "Staff Dashboard"
    elif role == 'ceo':
        return "CEO Dashboard"
    return ""

def add_contact(name, age, location, notes, role):
    if role not in ['staff', 'ceo']:
        return "Access denied. Only staff or CEO can add contacts."
    if not all([name, age, location, notes]):
        return "All fields must be filled to add a contact."

    conn = sqlite3.connect('disaster_relief.db')
    cursor = conn.cursor()
    cursor.execute("INSERT INTO contacts (name, age, location, notes) VALUES (?, ?, ?, ?)",
                   (name, age, location, notes))
    conn.commit()
    conn.close()
    return "Contact added successfully!"

def view_contacts(role):
    if role not in ['staff', 'ceo']:
        return "Access denied. Only staff or CEO can view contacts."
    conn = sqlite3.connect('disaster_relief.db')
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM contacts")
    rows = cursor.fetchall()
    conn.close()
    return rows

def assign_location(volunteer_name, location):
    if not volunteer_name or not location:
        return "Please provide both your name and a location."
    assignments[volunteer_name] = location
    return f"{volunteer_name} assigned to {location}"

def view_my_assignment(volunteer_name):
    location = assignments.get(volunteer_name)
    return f"{volunteer_name} is assigned to: {location}" if location else f"{volunteer_name} has not been assigned yet."

def view_all_assignments(_):
    if not assignments:
        return "No volunteers have been assigned yet."
    return [f"{name} → {loc}" for name, loc in assignments.items()]

# --- Gradio Interface ---

with gr.Blocks() as app:
    gr.Markdown("# Disaster Relief Volunteer Management System")

    user_role = gr.State()
    current_user = gr.State()
    is_logged_in = gr.State(value=False)

    with gr.Column(visible=True) as login_register_section:
        gr.Markdown("## Register")
        reg_username = gr.Textbox(label="Username")
        reg_password = gr.Textbox(label="Password", type="password")
        reg_role = gr.Textbox(label="Role (volunteer, staff, ceo)")
        reg_button = gr.Button("Register")
        reg_output = gr.Textbox(label="Registration Status")

        gr.Markdown("## Login")
        login_username = gr.Textbox(label="Username")
        login_password = gr.Textbox(label="Password", type="password")
        login_button = gr.Button("Login")
        login_output = gr.Textbox(label="Login Status")

        reg_button.click(register_user,
                         inputs=[reg_username, reg_password, reg_role],
                         outputs=reg_output)

        login_button.click(
            login_user,
            inputs=[login_username, login_password],
            outputs=[login_output, user_role, current_user, is_logged_in]
        )

    with gr.Column(visible=False) as main_app:
        dashboard_output = gr.Textbox(label="Dashboard")

        user_role.change(show_dashboard, inputs=[user_role], outputs=dashboard_output)

        logout_button = gr.Button("Logout")

        # --- Volunteer Section ---
        with gr.Column(visible=False) as volunteer_section:
            gr.Markdown("## Volunteer Assignment Section")
            assign_name = gr.Textbox(label="Your Full Name")
            assign_location_input = gr.Textbox(label="Enter Location to Assign Yourself")
            assign_button = gr.Button("Assign Me")
            assign_status = gr.Textbox(label="Assignment Status")

            view_my_button = gr.Button("View My Assignment")
            view_my_output = gr.Textbox(label="Your Current Assignment")

            assign_button.click(assign_location,
                                inputs=[assign_name, assign_location_input],
                                outputs=assign_status)

            view_my_button.click(view_my_assignment,
                                 inputs=[assign_name],
                                 outputs=view_my_output)

        # --- Shared Assignment Viewer ---
        with gr.Column(visible=True) as assignment_view_section:
            gr.Markdown("## Volunteer Assignment List")
            view_all_button = gr.Button("View All Assignments")
            all_assignments_output = gr.Textbox(label="All Volunteer Assignments", lines=10)

            view_all_button.click(view_all_assignments,
                                  inputs=[user_role],
                                  outputs=all_assignments_output)

        # --- Staff & CEO Section ---
        with gr.Column(visible=False) as staff_ceo_section:
            gr.Markdown("## Staff & CEO Contact Management")
            name = gr.Textbox(label="Contact Name")
            age = gr.Number(label="Age")
            location = gr.Textbox(label="Relief Location")
            donations = gr.Textbox(label="Donation Provided")
            submit_button = gr.Button("Add Contact")
            contact_output = gr.Textbox(label="Contact Status")

            submit_button.click(add_contact,
                                inputs=[name, age, location, donations, user_role],
                                outputs=contact_output)

            view_button = gr.Button("View Contacts")
            view_output = gr.Dataframe()

            view_button.click(view_contacts,
                              inputs=[user_role],
                              outputs=view_output)

    # --- Show/Hide Logic ---

    def update_visibility(is_logged_in, role):
        return {
            login_register_section: gr.update(visible=not is_logged_in),
            main_app: gr.update(visible=is_logged_in),
            volunteer_section: gr.update(visible=(role == 'volunteer')),
            staff_ceo_section: gr.update(visible=(role in ['staff', 'ceo']))
        }

    is_logged_in.change(update_visibility,
                        inputs=[is_logged_in, user_role],
                        outputs=[
                            login_register_section, main_app,
                            volunteer_section, staff_ceo_section
                        ])

    def logout():
        return False, None, None, "", "", ""

    logout_button.click(
        logout,
        outputs=[is_logged_in, user_role, current_user, login_output, reg_output, dashboard_output]
    )

app.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://0083ead50cb848b44d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
